# DM-2 Model Documentation

#### Note: 
In this sample code we make use of `runBlocking` to access suspending methods.  
However, <font color=orange>you should avoid blocking calls (runBlocking) within your code to prevent deadlocks and performance impacts</font>.  
Instead change the respecting methods to suspend functions and only wrap your main method in one single blocking call, e.g.
```kotlin
@JvmStatic
fun main(args: Array<String>) = runBlocking {
	// your method calls
}
```


<br>First, before you can use any imported class, load the (maven) dependencies

In [ ]:
// customize the repository from which we fetch the (maven) dependency
%classpath config resolver com.github https://jitpack.io
// add the dependency to classpath
%classpath add mvn com.github.hotzkow explorationModel master-SNAPSHOT jar
// we can check the classpath via
ClasspathManager.getJars()

## Custom Model Extension

Extending the default model allows you to overwrite any open function like the id computation or the criteria when any widget is considered 'interactive'.  
Moreover, you may add additional properties depending on your specific exploration strategy.

Note: that any *new custom properties* are not persisted on model dump, meaning if this property cannot be recomputed at initialization you need to persit these properties on your own.

In [ ]:
import kotlinx.coroutines.runBlocking
import org.droidmate.deviceInterface.exploration.UiElementPropertiesI
import org.droidmate.explorationModel.config.ModelConfig
import org.droidmate.explorationModel.*
import org.droidmate.explorationModel.factory.*
import org.droidmate.explorationModel.interaction.*

class CustomWidget(properties: UiElementPropertiesI,
                   parentId: ConcreteId?): Widget(properties,parentId){
	override fun toString(): String = super.toString().replace("Widget","CustomWidget")
}

class CustomState(_widgets: Collection<CustomWidget>, isHomeScreen: Boolean = false): State<CustomWidget>(_widgets,isHomeScreen){
	override fun toString(): String = super.toString().replace("State","CustomState")
}

class CustomModel(
	override val config: ModelConfig,
	override val stateProvider: StateFactory<CustomState, CustomWidget>,
	override val widgetProvider: WidgetFactory<CustomWidget>
) : AbstractModel<CustomState,CustomWidget>(){
	override fun toString(): String = super.toString().replace("Model","CustomModel")
}

class CustomModelProvider: ModelProvider<CustomModel>() {

	val stateProvider = object : StateProvider<CustomState,CustomWidget>(){
		override fun init(widgets: Collection<CustomWidget>, isHomeScreen: Boolean): CustomState = CustomState(widgets, isHomeScreen)
	}
    
	@Suppress("MemberVisibilityCanBePrivate")
	val widgetProvider = object : WidgetProvider<CustomWidget>(){
		override fun init(properties: UiElementPropertiesI, parentId: ConcreteId?): CustomWidget = CustomWidget(properties,parentId)
	}

	override fun create(config: ModelConfig): CustomModel
	= CustomModel(config = config, stateProvider = stateProvider, widgetProvider = widgetProvider)
}

Usually you would simple pass your `CustomModelProvider` to the explore method DM-2's `ExplorationAPI`.  
However for the following demonstrations we want to manually create a 'dummy' model.

In [ ]:
import com.natpryce.konfig.ConfigurationMap

object DemoModel{
  // first initialize our model provider
  val mp = CustomModelProvider()
  val model get() = mp.get()
  
  // before we can create an empty model we have to initialize the model config
  // usually DM-2 would do this for us
  val customCfg = ConfigurationMap(mapOf(
    "Output.outputDir" to "/tmp/DM-2",
    // to prevent removal of previously stored models (per default $outputDir/models is cleared on initialization)
//       "ModelProperties.path.cleanDirs" to "false",
    "ModelProperties.dump.sep" to ","
  ))
  
  init{
    
    // create the configuration according to your app-name, we can pass a custom configuration [cfg], e.g. to customize the path where the model is stored
    // if [isLoadC] is false all (parent) directories will be created automatically but not if it is true, since that could overwrite existing files
    val cfg = ModelConfig("TestModel", isLoadC = false, cfg = customCfg)
    // initialize the config in the model provider
    mp.config = cfg
    // the path where the model is going to be stored
    println(mp.config.baseDir)
    beakerx["modelpath"] = mp.config.baseDir.toString()
  }

  // let us add some widget and state objects to create a demo model
  // adding/getting states/widgets is asynchronous, therefore it can only be called from a corourtine scope
  suspend fun create(){
    val se = model.emptyState
    // the model uses [model.addState(se)], but this method is package private
    mp.stateProvider.addState(se)
    // the states are organized as Set meaning we won't have any duplicate states, but the [addState] property will show how often this method was called
    mp.stateProvider.addState(se)
    val we = model.emptyWidget
    //FIXME lazy create does not work in jupyter?
  //     val state by mp.stateProvider.lazyCreate(Pair(listOf(we),false))
    val state = mp.stateProvider.create(Pair(listOf(we),false))
    mp.stateProvider.addState(state)

    // create a dummy exploration trace with a single action leading to our second state, containing one widget
    val et = model.initNewTrace()
    et.update(EmptyActionResult,state)
  }
}

runBlocking{

  // we start with an empty model
  val model = DemoModel.model
  println(model)
  println()

  DemoModel.create()
  // we now should have two states and one widget in the model
  val states = runBlocking{ model.getStates() }
  println("states = \n" +                                               
          "\t${states.joinToString(separator = ",\n\t")}; \n\n" +
          "model = $model")
}

When inspecting the states of our demo model we see that one of them contains a widget.
However, the Model.toString() method will not show this widget.  
This is caused by the fact that we 'manually' injected the elements and were not able to call the internal model update function which would have increased this counter.

In any case the `addState` and `addWidget` properties only tell us how often the internal add functions were called.  
To determine how many states and elements a model really has we need to call `getStates` and `getWidgets` and count the number of elements. Since this call may be rather expensive for big models this is not done by default and should be avoided as long as this information is not necessary.

In [ ]:
runBlocking{
  val numWidgets = DemoModel.model.getWidgets().count()
  val numStates = DemoModel.model.getStates().count()
  println("#widgets = $numWidgets\n" +
          "#states  = $numStates")
}

##  Persisting a Model

By default DM-2 will store the model to the confiured path after each action (i.e. if ModelProperties.dump.onEachAction=true).

Alternativel the model can be stored at any arbitrary time by invoking the method `dumpModel(config)` on the model object.

In [ ]:
runBlocking{ 
  DemoModel.model.dumpModel(DemoModel.mp.config)
}

In [ ]:
%%python

from beakerx.object import beakerx

# there should appear two csv files and one of them contains one widget entry
import os
print(os.listdir(beakerx.modelpath+"/states"))

import pandas as pd

# beakerx.modelpath = tmp/DM-2/model/TestModel
print( pd.read_csv(beakerx.modelpath+"/states/c7468efa-44dc-6869-9ca9-0bb9a2d478e3_e835e6fc-78b8-9974-bd98-63bbd6b0a9ee.csv") )

## Model Loading

To load a model, you first need to invoke the respecive method of the modelparser with your (custom) modelprovider.

In [ ]:
import org.droidmate.explorationModel.config.ModelConfig
import org.droidmate.explorationModel.retention.loading.*
import kotlinx.coroutines.*

 
runBlocking{
  // Note: the loaded model will only contain states which are listed in the exploration trace file
  val mpl = CustomModelProvider()
  mpl.config = ModelConfig("TestModel", isLoadC = true, cfg = DemoModel.customCfg)
  val m = ModelParser.loadModel(modelProvider = mpl, sequential = true)
  println(m)

  val numWidgets = m.getWidgets().count()
  val numStates = m.getStates().count()
  println("#widgets = $numWidgets\n" +
          "#states  = $numStates")

}